In [1]:
import spacy
import newspaper
import pandas as pd
import json
from datetime import datetime
from bs4 import BeautifulSoup
import requests

In [2]:
# Charger le modèle de traitement du langage naturel français de SpaCy
nlp = spacy.load("fr_core_news_sm")

def extract_triplets(article_text):
    # Initialiser une liste pour stocker les triplets
    triplets = []

    # Analyser le texte de l'article avec SpaCy
    doc = nlp(article_text)

    # Parcourir les phrases du document
    for sentence in doc.sents:
        # Analyser la syntaxe de la phrase
        parsed_sentence = nlp(sentence.text)

        # Extraire les relations sujet-verbe-objet
        for token in parsed_sentence:
            if token.dep_ == "nsubj" and token.head.pos_ == "VERB":
                sujet = token.text
                predicat = token.head.lemma_
                objet = None
                for child in token.head.children:
                    if child.dep_ == "obj":
                        objet = child.text
                        break

                # Ajouter le triplet à la liste
                triplet = f"{sujet} {predicat} {objet}"
                triplets.append(triplet)

    return triplets

def extract_date_from_content(content):
    # Fonction pour extraire une date à partir du contenu de l'article
    possible_dates = []
    for line in content.split('\n'):
        try:
            date = datetime.strptime(line.strip(), "%Y-%m-%d")
            possible_dates.append(date)
        except ValueError:
            continue
    return possible_dates[0] if possible_dates else None

def extract_date_from_meta(soup):
    # Fonction pour extraire une date à partir des balises meta de l'article
    date_meta_tags = ['article:published_time', 'date', 'DC.date.issued']
    for tag in date_meta_tags:
        meta = soup.find('meta', attrs={'property': tag}) or soup.find('meta', attrs={'name': tag})
        if meta and meta.get('content'):
            try:
                return datetime.fromisoformat(meta['content'])
            except ValueError:
                continue
    return None

def scrape_articles_from_links(file_path):
    # Lire le fichier Excel contenant les liens des articles
    df = pd.read_excel(file_path)

    article_data = []

    for index, row in df.iterrows():
        site_url = row['site_url']
        article_url = row['article_url']

        try:
            # Initialiser l'article avec Newspaper
            article = newspaper.Article(article_url)
            article.download()
            article.parse()

            # Extraire les informations de l'article
            titre = article.title
            contenu = article.text
            date_publication = article.publish_date
            auteurs = article.authors

            if date_publication is None:
                # Tenter de récupérer la date à partir du contenu de l'article
                date_publication = extract_date_from_content(contenu)

                # Tenter de récupérer la date à partir des métadonnées
                if date_publication is None:
                    html = requests.get(article_url).text
                    soup = BeautifulSoup(html, 'html.parser')
                    date_publication = extract_date_from_meta(soup)

            # Extraire les triplets du contenu de l'article
            triplets = extract_triplets(contenu)

            # Ajouter les données de l'article à la liste
            article_data.append({
                "site_url": site_url,
                "titre": titre,
                "contenu": contenu,
                "date": date_publication.isoformat() if date_publication else None,
                "auteurs": ", ".join(auteurs),
                "triplets": triplets
            })

        except Exception as e:
            print(f"Erreur lors du traitement de l'article de {article_url}: {e}")
            continue

    # Enregistrer les données dans un fichier JSON
    with open('articles.json', 'w', encoding='utf-8') as f:
        json.dump(article_data, f, ensure_ascii=False, indent=4)

    print("Les données ont été enregistrées dans le fichier 'articles.json'.")

# Chemin vers le fichier Excel contenant les liens des articles
file_path = "C:\\Users\\WANE\\Desktop\\Mémoire\\article_links.xlsx"

# Lancer le processus de scraping
scrape_articles_from_links(file_path)


Erreur lors du traitement de l'article de https://kawtef.com/deces-de-la-maman-de-mbaye-dieye-faye-youssou-ndour-console-son-meilleur-ami_116030.html: Article `download()` failed with HTTPSConnectionPool(host='kawtef.com', port=443): Read timed out. on URL https://kawtef.com/deces-de-la-maman-de-mbaye-dieye-faye-youssou-ndour-console-son-meilleur-ami_116030.html
Erreur lors du traitement de l'article de https://kawtef.com/serie-decheances-saison-2-episode-37-bande-annonce_116023.html: Article `download()` failed with HTTPSConnectionPool(host='kawtef.com', port=443): Read timed out. on URL https://kawtef.com/serie-decheances-saison-2-episode-37-bande-annonce_116023.html
Erreur lors du traitement de l'article de https://www.dakaractu.com/New-Delhi-Le-president-Diomaye-exile-le-General-Souleymane-Kande-chef-d-Etat-major-de-l-armee-de-terre-comme-attache_a249076.html: HTTPSConnectionPool(host='www.dakaractu.com', port=443): Max retries exceeded with url: /New-Delhi-Le-president-Diomaye-exi